In [1]:
import numpy
import pandas as pd
from IPython.display import display
import matplotlib
import matplotlib.pylab as plt
matplotlib.style.use("ggplot")
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

print("test")

test
